In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import librosa
import librosa.display
import IPython.display as ipd
from itertools import cycle
from tqdm import tqdm

# Configuration

In [21]:
class Configuration:

  recompute_dataframes=False

  class Paths:
    train_audio_folders = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'
    train_audio_files='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio/*/*.ogg'
    unlabeld_soundspaces='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/unlabeled_soundscapes'
    taxonomy_birds = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/eBird_Taxonomy_v2021.csv'
    meta_data='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_metadata.csv'
    sample_submission='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/sample_submission.csv'


  class ClassInfo:
    class_names=sorted(os.listdir('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'))
    num_classes=len(class_names)
    labels=list(range(num_classes))
    labels_with_names = dict(zip(labels,class_names))

config=Configuration()

# Extracting audio signals and sample rates for audio processing

In [22]:
# Code for extracting the audio signal and sample rate of single example of each bird species
meta_data=pd.read_csv(config.Paths.meta_data)
meta_data['filepath'] = config.Paths.train_audio_folders+'/'+meta_data.filename
if config.recompute_dataframes:
  birds=meta_data['primary_label'].unique()
  df= meta_data.set_index('primary_label')
  bird_species=[]
  audio_signal=[]
  sample_rate=[]
  audio_data={}
  for bird in tqdm(birds, desc='Loading audio signal and sample rate'):

    path = df.loc[bird].iloc[0,:]['filepath']
    audio, sr = librosa.load(path)
    bird_species.append(bird)
    audio_signal.append(audio)
    sample_rate.append(sr)

  audio_data={'bird_species':bird_species,'audio_signal':audio_signal,'sample_rate':sample_rate}
  pd.DataFrame(audio_data).to_json('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/audio_data.json')
else:
  audio_data = pd.read_json('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/audio_data.json')

In [ ]:
# Code for extracting the audio signal and sample rate of all the examples of each bird species
bird_species=[]
audio_signal=[]
sample_rate=[]
audio_data={}
audio, sr = 0,0
for bird in birds:
  if bird.startswith('c'):

    for i in tqdm(range(df.loc[bird].shape[0]),desc=f'Loading audio signal and sample rate for {bird}'):
      path = df.loc[bird].iloc[0,:]['filepath']
      audio, sr = librosa.load(path)
      bird_species.append(bird)
      audio_signal.append(audio)
      sample_rate.append(sr)

    globals()[f'audio_data_{bird[0]}']={'bird_species':bird_species,'audio_signal':audio_signal,'sample_rate':sample_rate}
  else:
    continue